In [ ]:
import os
from matplotlib import pyplot as plt
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.optimizers import Adam

In [ ]:
from google.colab import files
data = files.upload()

Saving archive.zip to archive (1).zip


In [ ]:
import zipfile
with zipfile.ZipFile('archive.zip', 'r') as zip_ref:
  zip_ref.extractall('face-mask-dataset')

In [ ]:
print(os.listdir('face-mask-dataset/data'))

['with_mask', 'without_mask']


# Preprocessing

In [ ]:
image_size = (224, 224)
batch_size = 32

datagen = ImageDataGenerator(

    rescale=1.0/255.0,
    validation_split=0.2
)



# Divide the images into training and validation

In [ ]:
train_gen= datagen.flow_from_directory(
    "face-mask-dataset/data",
    target_size=image_size,
    batch_size=batch_size,
    class_mode='binary',
    subset='training',
)

val_gen= datagen.flow_from_directory(
    'face-mask-dataset/data',
    target_size=image_size,
    batch_size=batch_size,
    class_mode='binary',
    subset='validation',
)

Found 6043 images belonging to 2 classes.
Found 1510 images belonging to 2 classes.


In [ ]:
print('Train batches', len(train_gen))
print('Validation batches', len(val_gen))

Train batches 189
Validation batches 48


# Use pre-trained model "MobileNetV2

In [ ]:
base_model= MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

#Freeze the layers
for layer in base_model.layers:
  layer.trainable= False

#Add a layer for classification
x= base_model.output
x= GlobalAveragePooling2D()(x)
x= Dense(128, activation='relu')(x)
predictions= Dense(1, activation='sigmoid', trainable= True)(x)

#Define the final model afterr adding the layer
model= Model(inputs= base_model.input, outputs= predictions)

#preparing the model for training
model.compile(optimizer= Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

#train the model
new_model= model.fit(train_gen, validation_data= val_gen, epochs=5)



Epoch 1/5
189/189 ━━━━━━━━━━━━━━━━━━━━ 318s 2s/step - accuracy: 0.9538 - loss: 0.1096 - val_accuracy: 0.9841 - val_loss: 0.0404
Epoch 2/5
189/189 ━━━━━━━━━━━━━━━━━━━━ 300s 2s/step - accuracy: 0.9949 - loss: 0.0154 - val_accuracy: 0.9834 - val_loss: 0.0454
Epoch 3/5
189/189 ━━━━━━━━━━━━━━━━━━━━ 307s 2s/step - accuracy: 0.9973 - loss: 0.0098 - val_accuracy: 0.9894 - val_loss: 0.0362
Epoch 4/5
189/189 ━━━━━━━━━━━━━━━━━━━━ 298s 2s/step - accuracy: 0.9989 - loss: 0.0045 - val_accuracy: 0.9901 - val_loss: 0.0410
Epoch 5/5
189/189 ━━━━━━━━━━━━━━━━━━━━ 308s 2s/step - accuracy: 0.9987 - loss: 0.0054 - val_accuracy: 0.9868 - val_loss: 0.0342
